In [ ]:
cat = "Comics_characters"

catType = {"Airports":"Infrastructure","Comics_characters":"Agent","Artist":"Animal","Astronauts":'Animal',"Building":""}
k = 100
SPARQL_GET_LISTS = f'''
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbc: <http://dbpedia.org/resource/Category:>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dct: <http://purl.org/dc/terms/>

SELECT DISTINCT * WHERE {{
?a dcterms:subject/skos:broader dbc:{cat} .
?a a dbo:{catType[cat]}
}}limit {k}
'''

